In [1]:
import requests as rq
from bs4 import BeautifulSoup as bs
from csv import writer
import re
import pandas as pd
import numpy as np
import csv

In [2]:
def get_page(page):
    page= str(page)
    # Construct the URL
    books_page_url = 'https://www.audible.in/search?feature_six_browse-bin=22212175031&node=21881793031&pageSize=25&sort=&page=' + page
    print(books_page_url)
    
    # Get the HTML page content using requests
    response = rq.get(books_page_url)
    
    # Ensure that the reponse is valid
    if response.status_code != 200:
        print('Status code:', response.status_code)
        raise Exception('Failed to fetch web page ' + books_page_url)
    
    # Construct a beautiful soup document
    doc = bs(response.text)
    
    return doc

In [3]:
page_1= get_page(1)

https://www.audible.in/search?feature_six_browse-bin=22212175031&node=21881793031&pageSize=25&sort=&page=1


In [4]:
book_contents= page_1.find_all('li', class_='bc-list-item productListItem')

In [5]:
def get_book_names(book_contents):
    book_names= []
    for tag in book_contents:
            a_tag_name= tag.h3.find_all('a', recursive=False)
            book_name= a_tag_name[0].text.strip()
            book_names.append(book_name)
    return book_names

In [6]:
get_book_names(book_contents)

['Build',
 'Start with Why',
 '$100M Offers',
 'The Design of Everyday Things',
 'Androids',
 'Think and Grow Rich',
 'Hooked: How to Build Habit-Forming Products',
 'The Warren Buffett Way',
 'The 80/20 Principle',
 'Leaders Eat Last',
 'The Personal MBA: Master the Art of Business',
 'Trading in the Zone',
 'Humour, Seriously',
 'The Great Mental Models',
 'Skill It, Kill It',
 'The Lean Startup',
 'The Five Dysfunctions of a Team',
 'Range',
 'Nudge: The Final Edition',
 'Measure What Matters']

In [7]:
def get_book_links(book_contents):
    base_url='https://www.audible.in'
    book_links=[]
    for tag in book_contents:
        a_tag_name= tag.h3.find_all('a', recursive=False)
        url= a_tag_name[0]['href'].strip()
        book_link= base_url+url
        book_links.append(book_link)
    return book_links

In [8]:
books_url=get_book_links(book_contents)
books_url

['https://www.audible.in/pd/Build-Audiobook/B09MSDNL75',
 'https://www.audible.in/pd/Start-with-Why-Audiobook/B09J5J1PTZ',
 'https://www.audible.in/pd/100M-Offers-Audiobook/B09BK615JG',
 'https://www.audible.in/pd/The-Design-of-Everyday-Things-Audiobook/B07L5T1Q55',
 'https://www.audible.in/pd/Androids-Audiobook/B0B6Y6Z763',
 'https://www.audible.in/pd/Think-and-Grow-Rich-Audiobook/B07BN7HM3H',
 'https://www.audible.in/pd/Hooked-How-to-Build-Habit-Forming-Products-Audiobook/B079TLX3QQ',
 'https://www.audible.in/pd/The-Warren-Buffett-Way-Audiobook/B079J31PHQ',
 'https://www.audible.in/pd/The-80-20-Principle-Audiobook/B079DG92WG',
 'https://www.audible.in/pd/Leaders-Eat-Last-Audiobook/B0764KZ64Y',
 'https://www.audible.in/pd/The-Personal-MBA-Master-the-Art-of-Business-Audiobook/B079TN1LCT',
 'https://www.audible.in/pd/Trading-in-the-Zone-Audiobook/B07BTC3K6T',
 'https://www.audible.in/pd/Humour-Seriously-Audiobook/0241991382',
 'https://www.audible.in/pd/The-Great-Mental-Models-Audiobook

In [9]:
len(books_url)

20

In [10]:
def get_written_by(book_contents):
    written_by=[]
    for tag in book_contents:
        author_tag= tag.find('li', class_='bc-list-item authorLabel')
        try:
            auth_tag = author_tag.find('a')
            author = auth_tag.text.strip()
            written_by.append(author)
        except AttributeError:
            written_by.append(None)
    return written_by

In [11]:
get_written_by(book_contents)

['Tony Fadell',
 'Simon Sinek',
 'Alex Hormozi',
 'Don Norman',
 'Chet Haase',
 'Napoleon Hill',
 'Nir Eyal',
 'Robert Hagstrom',
 'Richard Koch',
 'Simon Sinek',
 'Josh Kaufman',
 'Mark Douglas',
 'Jennifer Aaker',
 'Shane Parrish',
 'Ronnie Screwvala',
 'Eric Ries',
 'Patrick Lencioni',
 'David Epstein',
 'Richard H. Thaler',
 'John Doerr']

In [12]:
def get_cover_img(book_contents):
    cover_img=[]
    for tag in book_contents:
        img_tag= tag.find_all('img', class_='bc-pub-block bc-image-inset-border js-only-element')
        try:
            #price_tag = img_tag.find('span', class_='bc-text bc-size-base bc-color-base')
            book_image_url= img_tag[0]['src'].strip()
            cover_img.append(book_image_url)
        except AttributeError:
            cover_img.append(None)
    return cover_img

In [13]:
get_cover_img(book_contents)

['https://m.media-amazon.com/images/I/41TdnZ8HOeL._SL500_.jpg',
 'https://m.media-amazon.com/images/I/41Px2q4eSiL._SL500_.jpg',
 'https://m.media-amazon.com/images/I/51DbY4as4EL._SL500_.jpg',
 'https://m.media-amazon.com/images/I/51Dl6lXXesL._SL500_.jpg',
 'https://m.media-amazon.com/images/I/51e-6jSOitL._SL500_.jpg',
 'https://m.media-amazon.com/images/I/51vKha04DuL._SL500_.jpg',
 'https://m.media-amazon.com/images/I/51o-DT4Z33L._SL500_.jpg',
 'https://m.media-amazon.com/images/I/51lmFVyRNjL._SL500_.jpg',
 'https://m.media-amazon.com/images/I/51uLXLLL3nL._SL500_.jpg',
 'https://m.media-amazon.com/images/I/41+s+NeZK9L._SL500_.jpg',
 'https://m.media-amazon.com/images/I/51crV7aDATL._SL500_.jpg',
 'https://m.media-amazon.com/images/I/51+8mBt4k7L._SL500_.jpg',
 'https://m.media-amazon.com/images/I/419Gvy5zK7L._SL500_.jpg',
 'https://m.media-amazon.com/images/I/51frmShzuIL._SL500_.jpg',
 'https://m.media-amazon.com/images/I/517RdWh2VgL._SL500_.jpg',
 'https://m.media-amazon.com/images/I/51

def get_rating(book_contents):
    rating=[]
    for tag in book_contents:
        star_tag= tag.find('li', class_='bc-list-item ratingsLabel')
        try:
            rating_tag = star_tag.find('span', class_='bc-text bc-pub-offscreen').text.strip()
            rating.append(rating_tag[0])
        except AttributeError:
            rating.append(None)
    return rating

In [14]:
#get all book_urls
def parse_pages(end_page):
  
    all_links=[]
    for page in range (1,end_page):
        page_x = get_page(page)
        book_contents = page_x.find_all('li', class_='bc-list-item productListItem')
        for link in get_book_links(book_contents):
            all_links.append(link)
            
        page = page + 1
        
    return all_links

In [20]:
Books_links=(parse_pages(6))

https://www.audible.in/search?feature_six_browse-bin=22212175031&node=21881793031&pageSize=25&sort=&page=1
https://www.audible.in/search?feature_six_browse-bin=22212175031&node=21881793031&pageSize=25&sort=&page=2
https://www.audible.in/search?feature_six_browse-bin=22212175031&node=21881793031&pageSize=25&sort=&page=3
https://www.audible.in/search?feature_six_browse-bin=22212175031&node=21881793031&pageSize=25&sort=&page=4
https://www.audible.in/search?feature_six_browse-bin=22212175031&node=21881793031&pageSize=25&sort=&page=5


In [21]:
len(Books_links)

100

In [22]:
def parse_pages_ranged(end_page):
    all_page_contents = {
            'Title':[],
            'Author':[],
            'Image_URL':[],
            'Book_URL':[],
            }    
    for page in range (1,end_page):
        page_x = get_page(page)
        book_contents = page_x.find_all('li', class_='bc-list-item productListItem')
        all_page_contents['Title'] += get_book_names(book_contents)
        all_page_contents['Author'] += get_written_by(book_contents)
        all_page_contents['Image_URL'] += get_cover_img(book_contents)
        all_page_contents['Book_URL'] += get_book_links(book_contents)
        
        page = page + 1
    return all_page_contents

In [25]:
df1= pd.DataFrame(parse_pages_ranged(6))
df1


https://www.audible.in/search?feature_six_browse-bin=22212175031&node=21881793031&pageSize=25&sort=&page=1
https://www.audible.in/search?feature_six_browse-bin=22212175031&node=21881793031&pageSize=25&sort=&page=2
https://www.audible.in/search?feature_six_browse-bin=22212175031&node=21881793031&pageSize=25&sort=&page=3
https://www.audible.in/search?feature_six_browse-bin=22212175031&node=21881793031&pageSize=25&sort=&page=4
https://www.audible.in/search?feature_six_browse-bin=22212175031&node=21881793031&pageSize=25&sort=&page=5


,Title,Author,Image_URL,Book_URL
0,Build,Tony Fadell,https://m.media-amazon.com/images/I/41TdnZ8HOe...,https://www.audible.in/pd/Build-Audiobook/B09M...
1,Start with Why,Simon Sinek,https://m.media-amazon.com/images/I/41Px2q4eSi...,https://www.audible.in/pd/Start-with-Why-Audio...
2,$100M Offers,Alex Hormozi,https://m.media-amazon.com/images/I/51DbY4as4E...,https://www.audible.in/pd/100M-Offers-Audioboo...
3,The Design of Everyday Things,Don Norman,https://m.media-amazon.com/images/I/51Dl6lXXes...,https://www.audible.in/pd/The-Design-of-Everyd...
4,Androids,Chet Haase,https://m.media-amazon.com/images/I/51e-6jSOit...,https://www.audible.in/pd/Androids-Audiobook/B...
...,...,...,...,...
95,No Rules Rules,Reed Hastings,https://m.media-amazon.com/images/I/516qHOFsz1...,https://www.audible.in/pd/No-Rules-Rules-Audio...
96,Eat That Frog!,Brian Tracy,https://m.media-amazon.com/images/I/51LCpTzITa...,https://www.audible.in/pd/Eat-That-Frog-Audiob...
97,Prediction Machines,Ajay Agrawal,https://m.media-amazon.com/images/I/41kHE3IPma...,https://www.audible.in/pd/Prediction-Machines-...
98,15 Secrets Successful People Know About Time M...,Kevin Kruse,https://m.media-amazon.com/images/I/51QdssDxrT...,https://www.audible.in/pd/15-Secrets-Successfu...


In [26]:
def get_book_soup(book_url):
    response = rq.get(book_url)
    if response.status_code != 200:
        # print('Status code:', response.status_code)
        # raise Exception('Failed to fetch book page ' + book_url)
        pass
    else:
        doc = bs(response.text)
        return doc

In [27]:
def get_reviews(book_url): 
    Reviews=[] 
    try:  
        res = rq.get(book_url)
        doc2 = bs(res.text)
        # regexG=re.compile('.*bc-section listReviewsTabUS.*')
        # TagG=doc2.find('div', {'class':regexG})
        regexRatRev = re.compile('.*bc-row-responsive bc-spacing-top-s4 bc-spacing-s5.*')
        TagRatRev = doc2.find_all('div', {'class':regexRatRev})
        for Tag in TagRatRev:
            # regexRev = re.compile('.*bc-heading bc-color-base bc-pub-break-word bc-spacing-base bc-size-title1 bc-text-bold bc-text-emphasis.*')
            # results = Tag.findAll('h4', {'class':regexRev})
            regexRev = re.compile('.*bc-col-responsive USreviews0 bc-col-9.*|.*bc-col-responsive INreviews0 bc-col-9.*|.*bc-col-responsive UKreviews0 bc-col-9.*|.*bc-col-responsive AUreviews0 bc-col-9.*')
            results = Tag.findAll('div', {'class':regexRev})
            for result in results:
                review=result.text.replace('\n',' ')
                review=re.sub('\W+|\s+',' ', str(review))
                review=re.sub('\d+\s\w+\s\w+\s\w+\s\w+\s','', str(review))
                Reviews.append(review)
        return Reviews
    except AttributeError:
        return None
        pass

In [28]:
def get_rating_review(book_url):
    try:
        Rating=[]
        res = rq.get(book_url)
        doc2 = bs(res.text)
        # regexG=re.compile('.*bc-section listReviewsTabUS.*')
        # TagG=doc2.find('div', {'class':regexG})
        regexRatRev = re.compile('.*bc-row-responsive bc-spacing-top-s4 bc-spacing-s5.*')
        TagRatRev = doc2.find_all('div', {'class':regexRatRev})
        for Tag in TagRatRev:
            regexRat = re.compile('.*bc-text bc-pub-offscreen.*')
            results = Tag.find('span', {'class':regexRat})
            for result in results:
                rating =result.text.strip()[0:3]
                rating=re.sub('\D+$','', str(rating))
                Rating.append(rating)
        return Rating
    except AttributeError:
        pass
              
     

In [29]:
def get_type(book_url):
    try:
        doc = get_book_soup(book_url) 
        regex = re.compile('.*bc-link bc-size-base bc-color-link.*')
        results = doc.find_all('a', {'class':regex})
        book_type = [result.text.replace('\n','') for result in results][:1]   
        return book_type
    except AttributeError:
        pass
        
        
  

In [30]:
def get_resume(doc):
    try:
        resume_tag= doc.find(class_='bc-box bc-box-padding-none bc-spacing-s2')
        resume = resume_tag.find('span').text.strip()
        return resume 
    except AttributeError:
        # print('None')
        pass

In [31]:
All_books_info2=[]
count=0
for i in Books_links:
    count+=1
    print("book number",count)
    doc = get_book_soup(i)
    categorie=get_type(i)
    rating= get_rating_review(i)
    desc=get_resume(doc)
    reviews=get_reviews(i)
    rating
    All_books_info2.append({
            "Book_URL":i,
            "Type":categorie,
            "Description":desc,
            "Reviews": reviews,
            "Rating":rating

        })
All_books_info2
df2=pd.DataFrame(All_books_info2)
df2

book number 1
book number 2
book number 3
book number 4
book number 5
book number 6
book number 7
book number 8
book number 9
book number 10
book number 11
book number 12
book number 13
book number 14
book number 15
book number 16
book number 17
book number 18
book number 19
book number 20
book number 21
book number 22
book number 23
book number 24
book number 25
book number 26
book number 27
book number 28
book number 29
book number 30
book number 31
book number 32
book number 33
book number 34
book number 35
book number 36
book number 37
book number 38
book number 39
book number 40
book number 41
book number 42
book number 43
book number 44
book number 45
book number 46
book number 47
book number 48
book number 49
book number 50
book number 51
book number 52
book number 53
book number 54
book number 55
book number 56
book number 57
book number 58
book number 59
book number 60
book number 61
book number 62
book number 63
book number 64
book number 65
book number 66
book number 67
book

,Book_URL,Type,Description,Reviews,Rating
0,https://www.audible.in/pd/Build-Audiobook/B09M...,[Business & Careers],Brought to you by Penguin. Tony Fadell led the...,[ visionary only for those who seek vision cla...,"[4, 5, 4, 5, 5, 5, 5, 5, 1, 1, 3, 5, 5, 5, 5, ..."
1,https://www.audible.in/pd/Start-with-Why-Audio...,[Business & Careers],Brought to you by Penguin. Why are some peopl...,[ Too long and too many repetitions Honestly t...,"[3, 3, 5, 5, 5, 5, 5, 5, 5, 4, 2, 5, 4, 5, 5, ..."
2,https://www.audible.in/pd/100M-Offers-Audioboo...,[Business & Careers],I took home more in a year than the CEOs of Mc...,[],"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 2, 5, 5, 5, ..."
3,https://www.audible.in/pd/The-Design-of-Everyd...,None,"Design doesn't have to be complicated, which i...",[ Refund Audion isn t playing This audiobook i...,"[1, 5, 5, 3, 5, 4, 5, 5, 5, 5, 4, 5, 5, 5, 5, ..."
4,https://www.audible.in/pd/Androids-Audiobook/B...,[Computers & Technology],The fascinating inside story of how the Androi...,[ Rome wasn t built in a day so was Android It...,"[5, 4]"
...,...,...,...,...,...
95,https://www.audible.in/pd/No-Rules-Rules-Audio...,[Business & Careers],Hard work is irrelevant. Be radically honest. ...,[ Pronunciation issues Please check the pronun...,"[5, 5, 3, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
96,https://www.audible.in/pd/Eat-That-Frog-Audiob...,None,It’s time to stop procrastinating and get more...,[ Outstanding Life changing Narrator himself i...,"[5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 3, 4, 5, 5, 4, ..."
97,https://www.audible.in/pd/Prediction-Machines-...,[Business & Careers],"""What does AI mean for your business? Read thi...",[ A no fluff account of the times to come High...,"[5, 5, 5, 5, 5, 4, 5, 5, 3, 4, 2, 5, 2, 3, 4, ..."
98,https://www.audible.in/pd/15-Secrets-Successfu...,[Business & Careers],New York Times best-selling author Kevin Kruse...,[ Great content Lot of things that were spoken...,"[5, 5, 4, 5, 3, 4, 5, 5, 4, 5, 5, 5, 5, 5, 5, ..."


In [32]:
df2=df2.mask(df2.applymap(str).eq('[]'))

In [33]:
dfinal2=df1.merge(df2, on='Book_URL')
dfinal2.head()

,Title,Author,Image_URL,Book_URL,Type,Description,Reviews,Rating
0,Build,Tony Fadell,https://m.media-amazon.com/images/I/41TdnZ8HOe...,https://www.audible.in/pd/Build-Audiobook/B09M...,[Business & Careers],Brought to you by Penguin. Tony Fadell led the...,[ visionary only for those who seek vision cla...,"[4, 5, 4, 5, 5, 5, 5, 5, 1, 1, 3, 5, 5, 5, 5, ..."
1,Start with Why,Simon Sinek,https://m.media-amazon.com/images/I/41Px2q4eSi...,https://www.audible.in/pd/Start-with-Why-Audio...,[Business & Careers],Brought to you by Penguin. Why are some peopl...,[ Too long and too many repetitions Honestly t...,"[3, 3, 5, 5, 5, 5, 5, 5, 5, 4, 2, 5, 4, 5, 5, ..."
2,$100M Offers,Alex Hormozi,https://m.media-amazon.com/images/I/51DbY4as4E...,https://www.audible.in/pd/100M-Offers-Audioboo...,[Business & Careers],I took home more in a year than the CEOs of Mc...,NaN,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 2, 5, 5, 5, ..."
3,The Design of Everyday Things,Don Norman,https://m.media-amazon.com/images/I/51Dl6lXXes...,https://www.audible.in/pd/The-Design-of-Everyd...,None,"Design doesn't have to be complicated, which i...",[ Refund Audion isn t playing This audiobook i...,"[1, 5, 5, 3, 5, 4, 5, 5, 5, 5, 4, 5, 5, 5, 5, ..."
4,Androids,Chet Haase,https://m.media-amazon.com/images/I/51e-6jSOit...,https://www.audible.in/pd/Androids-Audiobook/B...,[Computers & Technology],The fascinating inside story of how the Androi...,[ Rome wasn t built in a day so was Android It...,"[5, 4]"


In [34]:
dfinal2.isna().sum().sort_values(ascending=False)

Reviews        24
Rating         18
Type           12
Description    12
Title           0
Author          0
Image_URL       0
Book_URL        0
dtype: int64

In [35]:
dfinal2.dropna(inplace=True)

In [36]:
dfinal2.reset_index(drop=True)

,Title,Author,Image_URL,Book_URL,Type,Description,Reviews,Rating
0,Build,Tony Fadell,https://m.media-amazon.com/images/I/41TdnZ8HOe...,https://www.audible.in/pd/Build-Audiobook/B09M...,[Business & Careers],Brought to you by Penguin. Tony Fadell led the...,[ visionary only for those who seek vision cla...,"[4, 5, 4, 5, 5, 5, 5, 5, 1, 1, 3, 5, 5, 5, 5, ..."
1,Start with Why,Simon Sinek,https://m.media-amazon.com/images/I/41Px2q4eSi...,https://www.audible.in/pd/Start-with-Why-Audio...,[Business & Careers],Brought to you by Penguin. Why are some peopl...,[ Too long and too many repetitions Honestly t...,"[3, 3, 5, 5, 5, 5, 5, 5, 5, 4, 2, 5, 4, 5, 5, ..."
2,Androids,Chet Haase,https://m.media-amazon.com/images/I/51e-6jSOit...,https://www.audible.in/pd/Androids-Audiobook/B...,[Computers & Technology],The fascinating inside story of how the Androi...,[ Rome wasn t built in a day so was Android It...,"[5, 4]"
3,Think and Grow Rich,Napoleon Hill,https://m.media-amazon.com/images/I/51vKha04Du...,https://www.audible.in/pd/Think-and-Grow-Rich-...,[Business & Careers],"""Think and Grow Rich"" is a motivational person...",[ Nice book but unstructured in Audible I am l...,"[1, 1, 5, 5, 4, 5, 3, 5, 5, 4, 5, 2, 4, 1, 4, ..."
4,The Warren Buffett Way,Robert Hagstrom,https://m.media-amazon.com/images/I/51lmFVyRNj...,https://www.audible.in/pd/The-Warren-Buffett-W...,[Biographies & Memoirs],Warren Buffett remains one of the most sought-...,[ To much information There s just a lot of in...,"[3, 4, 4, 3, 5, 5, 4, 4, 4, 5, 3, 5, 5, 5, 2, ..."
5,Leaders Eat Last,Simon Sinek,https://m.media-amazon.com/images/I/41+s+NeZK9...,https://www.audible.in/pd/Leaders-Eat-Last-Aud...,[Business & Careers],Brought to you by Penguin. Narrated by Simon S...,[ Interesting insights for Leaders I have been...,"[5, 5, 5, 4, 4, 5, 5, 4, 5, 4, 5, 5, 5, 4, 5, ..."
6,The Personal MBA: Master the Art of Business,Josh Kaufman,https://m.media-amazon.com/images/I/51crV7aDAT...,https://www.audible.in/pd/The-Personal-MBA-Mas...,[Business & Careers],"Audie Award Nominee, Business and Educational,...",[ Overall it s nice book The book is well narr...,"[4, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 4, 4, 5, 5, ..."
7,Trading in the Zone,Mark Douglas,https://m.media-amazon.com/images/I/51+8mBt4k7...,https://www.audible.in/pd/Trading-in-the-Zone-...,[Business & Careers],Douglas uncovers the underlying reasons for la...,[ Perfect listening i think this book makes fo...,"[5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 2, 5, 4, 5, 5, ..."
8,"Humour, Seriously",Jennifer Aaker,https://m.media-amazon.com/images/I/419Gvy5zK7...,https://www.audible.in/pd/Humour-Seriously-Aud...,[Business & Careers],Brought to you by Penguin. Anyone - even you!...,[ a perfect book for the current times super b...,"[4, 3, 4, 5, 5, 5, 4, 5, 5]"
9,"Skill It, Kill It",Ronnie Screwvala,https://m.media-amazon.com/images/I/517RdWh2Vg...,https://www.audible.in/pd/Skill-It-Kill-It-Aud...,[Business & Careers],"Ever wondered why CEOs, leaders and recruiters...",[ A must read book This is a must read book fo...,"[5, 5, 5, 5]"


In [37]:
dfinal2.shape

(51, 8)

In [38]:
dfinal2.to_csv("Business.csv")